# nb-model_exp-results-3-1 (h_pba_h)

In [1]:
import sys
import os
from os import sep
from os.path import dirname, realpath, exists, basename
from pathlib import Path
from glob import glob, iglob
import logging

def get_cwd(fname, subdir, crunch_dir=realpath(Path.home()) +sep +'crunch' +sep):
    """
    Convenience function to make a directory string for the current file based on inputs.
    Jupyter Notebook in Anaconda invokes the Python interpreter in Anaconda's subdirectory
    which is why changing sys.argv[0] is necessary. In the future a better way to do this
    should be preferred..
    """
    return crunch_dir +subdir +fname

def fix_path(cwd):
    """
    Convenience function to fix argv and python path so that jupyter notebook can run the same as
    any script in crunch.
    """
    sys.argv[0] = cwd
    module_path = os.path.abspath(os.path.join('..'))
    if module_path not in sys.path:
        sys.path.append(module_path)

fname = 'nb-model_exp-results-3-1.ipynb'
dir_name = 'model'
fix_path(get_cwd(fname, dir_name +sep))

import numpy as np
import pandas as pd

from ipywidgets import interact, interactive, fixed
from IPython.display import display

pd.set_option("display.max_rows", 300)
pd.set_option("display.max_columns", 50)

from common_util import MODEL_DIR, load_json, dump_json, rectify_json, load_df, str_now, makedir_if_not_exists, is_valid, isnt, compose, pd_split_ternary_to_binary, df_del_midx_level, midx_intersect, pd_get_midx_level, pd_rows, df_midx_restack
from common_util import NestedDefaultDict
from model.common import EXP_LOG_DIR, EXP_PARAMS_DIR, ASSETS, DATASET_DIR, XG_PROCESS_DIR, XG_DATA_DIR, XG_DIR, PYTORCH_MODELS_DIR, TRAIN_RATIO, EXPECTED_NUM_HOURS
from recon.viz import *
logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)

CRITICAL:root:script location: /home/kev/crunch/model/nb-model_exp-results-3-1.ipynb
CRITICAL:root:using project dir: /home/kev/crunch/


In [2]:
from model_util import StackedTCN
display(StackedTCN.get_receptive_field(2, 9, 8))
display(StackedTCN.get_receptive_field(3, 16, 2))

72

105

Prune the xg data down to the data of interest to use in further experiments.

In [3]:
def get_bench(pattern):
    ndd = NestedDefaultDict()
    for path in iglob(pattern, recursive=True):
        ndd[path.split('/')] = load_json(path)

    df_idx = [k[2] for k in ndd.keys()]
    df = pd.DataFrame.from_records([b['benchmark-hold'] for b in ndd.values()], index=df_idx)

    return df

In [4]:
def consolidate_results(pattern, bench_pattern=None):
    ndd = NestedDefaultDict()
    for path in iglob(pattern, recursive=True):
        ndd[path.split('/')] = load_json(path)
    df_keys = [(k[2], k[3], k[4], k[5], k[6]) for k in ndd.keys()]
    df_idx = pd.MultiIndex.from_tuples(df_keys, names=('asset', 'model', 'data', 'params', 'trial')) \
        .remove_unused_levels()
    df = pd.DataFrame.from_records(list(ndd.values()), index=df_idx) \
        .reset_index() \
        .groupby(['asset', 'model', 'params']).mean() # average across trials for each study

    if (is_valid(bench_pattern)):
        bench_df = get_bench(bench_pattern)
        mets = ['accuracy', 'precision', 'recall', 'f1']
        rets = ['profit', 'sharpe', 'cagr']
        for m in mets+rets:
            res_cols = [c for c in df.columns if (c.endswith(m))]
            bench_cols = [c for c in bench_df.columns if (c.endswith(m))]
            bench_col = bench_cols[0]

            for asset in bench_df.index:
#                 print(bench_df.loc[asset, bench_col])
#                 print(df.xs(asset, level=0, drop_level=False)[res_cols])
                df.xs(asset, level=0, drop_level=False).loc[:, res_cols] -= bench_df.loc[asset, bench_col]
#     df = df.style \
#         .background_gradient(subset=[c for c in df.columns if c.endswith(('accuracy', 'profit', 'sharpe', 'cagr'))], cmap='BuGn')
#     .set_sticky(axis="index") \
    return df

In [5]:
def agg_asset(res_df):
    agg_df = res_df.groupby(['model', 'params']).mean()
    return agg_df

In [6]:
def agg_model(res_df):
    agg_df = res_df.groupby(['params']).mean()
    return agg_df

In [7]:
def agg_asset_params(res_df):
    return res_df.groupby(['asset', 'params']).mean()

In [8]:
def add_style(df):
    df = df.style \
         .background_gradient(subset=[c for c in df.columns if c.endswith(('accuracy', 'f1', 'profit', 'sharpe', 'cagr'))], cmap='BuGn')
#     .set_sticky(axis="index") \
    return df

In [9]:
view = ('accuracy', 'f1', 'profit', 'sharpe', 'cagr')

def select_ends(df, ends=view):
    res_cols = [c for c in df.columns if (c.endswith(ends))]
#     for m in ends:
#         res_cols = [c for c in df.columns if (c.endswith(m))]
    return df.loc[:, res_cols]

In [10]:
def select_paramrange(df, r):
    params = map(str, r)
    return df.loc[(slice(None), slice(None), params), :]

In [11]:
def dropfirstmi(df):
    df.index = df.index.droplevel(0)
    return df

def gb_filter_monotonic(df, checkcol='val_binary_sharpe'):
    return dropfirstmi(df.reorder_levels(['asset', 'params', 'model']) \
            .groupby('params') \
            .apply(lambda g: g if (g[checkcol].is_monotonic_increasing) else None) \
            .sort_index())

subgt = lambda g, big, small='base': (g.xs(big, level=2) > g.xs(small, level=2)).all()
monotonic_partial = lambda g: (subgt(g, 'lnp') and subgt(g, 'np', 'lnp')) or (subgt(g, 'cnp') and subgt(g, 'np', 'cnp'))

def gb_filter_monotonic_partial(df, checkcol='val_binary_sharpe'):
    return dropfirstmi(df.reorder_levels(['asset', 'params', 'model']) \
            .groupby('params') \
            .apply(lambda g: g if (monotonic_partial(g[checkcol])) else None) \
            .sort_index())

nonneg = lambda g: (g.xs('np', level=2) > 0).all() and (g.xs('lnp', level=2) > 0).all()

def gb_filter_neg(df, checkcol='val_binary_sharpe'):
    return dropfirstmi(df.reorder_levels(['asset', 'params', 'model']) \
            .groupby('params') \
            .apply(lambda g: g if (nonneg(g[checkcol])) else None) \
            .sort_index())

def gb_filter_lf(df, checkcol='val_binary_longfreq'):
    return dropfirstmi(df.groupby('params') \
            .apply(lambda g: g if ((g[checkcol].between(.2, .8)).all()) else None))

In [12]:
def range_diff(df, target_range, base_range):
    tgt = df.loc[map(str, target_range)]
    return tgt - df.loc[map(str, base_range)].set_index(tgt.index)

In [13]:
# def get_params_per_dataset():
#     EXP_LOG_DIR

In [117]:
exp_base = basename(dirname(EXP_LOG_DIR))
sm_name = 'anp'

drange = '2009_2018'
xdata = 'h_pba_h'
ydata = 'ddir'
data_name = f'{drange}_{ydata}_{xdata}'
bench_data_name = f'{drange}_{ydata}'

path_train = f"{exp_base}/{sm_name}/**/{data_name}/**/train.json"
path_val =   f"{exp_base}/{sm_name}/**/{data_name}/**/val.json"
# path_test = f"{exp_base}/{sm_name}/**/{data_name}/**/test.json"

path_bench_train = f"{exp_base}/bench/*/{bench_data_name}/train.json"
path_bench_val = f"{exp_base}/bench/*/{bench_data_name}/val.json"
# path_bench_test = f"{exp_base}/bench/*/{bench_data_name}/test.json"

In [118]:
STUDY_START = 300
STUDY_END = 490
PRANGE = range(STUDY_START, STUDY_END)

# absolute results
dt = select_paramrange(consolidate_results(path_train), PRANGE);
dv = select_paramrange(consolidate_results(path_val), PRANGE);

# benchmark adjusted results
dt_adj = select_paramrange(consolidate_results(path_train, path_bench_train), PRANGE);
dv_adj = select_paramrange(consolidate_results(path_val, path_bench_val), PRANGE);

/home/kev/miniconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1787: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, val, pi)


## Results
Parameter sets >= 300

In [119]:
train_clf = [col for col in dt.columns if ('clf' in col)]
train_binary = [col for col in dt.columns if ('binary' in col)]
train_binary_sharpe = [col for col in train_binary if (col.endswith('sharpe'))]
trainb = train_binary_sharpe + ['train_binary_longfreq']

In [120]:
val_clf = [col for col in dv.columns if ('clf' in col)]
val_binary = [col for col in dv.columns if ('binary' in col)]
val_binary_sharpe = [col for col in val_binary if (col.endswith('sharpe'))]
val_binary_lf = ['val_binary_longfreq']
valb = val_binary_sharpe + ['val_binary_longfreq']

In [121]:
add_style(agg_model(agg_asset(dt_adj.loc[:, trainb])))

,train_binary_sharpe,train_binary_long_sharpe,train_binary_short_sharpe,train_binary_longfreq
params,,,,
300,2.027153,1.111858,1.161125,0.513198
301,2.263482,1.422204,1.162326,0.477865
302,0.719231,0.223811,0.236548,0.487157
303,2.165193,1.338260,1.107559,0.508937
304,2.184873,1.388259,1.089175,0.473485
305,0.545557,0.141481,0.067975,0.476681
306,1.216133,0.600305,0.534365,0.531020
307,1.980130,1.132152,1.092664,0.488636
308,1.374499,0.818442,0.546183,0.472139


In [122]:
add_style(agg_model(agg_asset(dv_adj.loc[:, valb])))

,val_binary_sharpe,val_binary_long_sharpe,val_binary_short_sharpe,val_binary_longfreq
params,,,,
300,0.392179,0.215605,0.140216,0.545776
301,0.447275,0.269051,0.181234,0.512451
302,0.577719,0.330273,0.301705,0.512573
303,0.414526,0.245385,0.147307,0.546753
304,0.437981,0.270577,0.163512,0.511230
305,0.521444,0.308685,0.262683,0.466553
306,0.473594,0.405275,0.044399,0.537720
307,0.483337,0.278530,0.221952,0.565674
308,0.181742,0.243315,-0.209697,0.505737


In [123]:
base_range = range(STUDY_START, STUDY_START+10)

### Compare study interventions globally

In [124]:
pavg = agg_asset_params(dv_adj.loc[:, val_binary_sharpe]).sort_index(level=[1, 0]).groupby('params').mean()

In [125]:
res = []
for start in range(STUDY_START+10, STUDY_END, 10):
    r = range(start, start+10)
    ser = range_diff(pavg, r, base_range).mean()
    ser.name = start
    res.append(ser)
res = pd.concat(res, axis=1).transpose()
add_style(res)

,val_binary_sharpe,val_binary_long_sharpe,val_binary_short_sharpe
310,-0.068024,-0.050964,-0.051192
320,-0.038134,-0.027419,-0.037561
330,-0.037521,-0.026268,-0.021850
340,-0.018985,-0.026520,-0.003797
350,-0.054884,-0.034359,-0.044911
360,-0.015984,-0.019561,-0.002514
370,-0.079226,-0.045086,-0.065500
380,-0.059626,-0.042053,-0.031796
390,-0.085621,-0.057051,-0.063214
400,-0.130112,-0.091659,-0.102945


In [126]:
# Information Ratio:
res = []
for start in range(STUDY_START+10, STUDY_END, 10):
    r = range(start, start+10)
    ser = range_diff(pavg, r, base_range).mean() / range_diff(pavg, r, base_range).std()
    ser.name = start
    res.append(ser)
res = pd.concat(res, axis=1).transpose()
add_style(res)

,val_binary_sharpe,val_binary_long_sharpe,val_binary_short_sharpe
310,-0.484745,-0.590402,-0.467267
320,-0.226261,-0.257934,-0.271353
330,-0.276110,-0.316813,-0.189928
340,-0.207495,-0.483889,-0.047471
350,-0.415606,-0.424729,-0.398281
360,-0.115413,-0.238494,-0.021412
370,-0.840015,-0.691859,-0.745752
380,-0.471795,-0.474890,-0.304892
390,-0.572887,-0.595650,-0.504585
400,-1.196438,-1.076031,-1.179679


### Compare study interventions for each asset

In [127]:
asset_pavg = dv_adj.loc[:, val_binary_sharpe].groupby(['params', 'asset']).mean()

In [128]:
res = []
for start in range(STUDY_START+10, STUDY_END, 10):
    r = range(start, start+10)
    ad = range_diff(asset_pavg, r, base_range).groupby('asset').mean()
    ad = ad.assign(start=start).set_index('start', append=True)
    res.append(ad)
res = pd.concat(res, axis=0).sort_index(level=[0, 1])
add_style(res)

In [129]:
res = []
for start in range(STUDY_START+10, STUDY_END, 10):
    r = range(start, start+10)
    #ad = range_diff(asset_pavg, r, base_range).groupby('asset').mean()
    ad = range_diff(asset_pavg, r, base_range).groupby('asset').mean() / range_diff(asset_pavg, r, base_range).groupby('asset').std()
    ad = ad.assign(start=start).set_index('start', append=True)
    res.append(ad)
res = pd.concat(res, axis=0).sort_index(level=[0, 1])
add_style(res)

In [130]:
#add_style(agg_asset_params(dv_adj.loc[:, val_binary_sharpe]).sort_index(level=[1, 0]))

## Recover Best Params By Asset

In [144]:
res = []
for start in range(STUDY_START+10, STUDY_END, 10):
    r = range(start, start+10)
    ad = range_diff(asset_pavg, r, base_range).groupby('asset').mean()
    ad = ad.assign(start=start).set_index('start', append=True)
    res.append(ad)
res = pd.concat(res, axis=0).sort_index(level=[0, 1])
topgroups = res.loc[res.groupby(['asset'])['val_binary_sharpe'].idxmax()]
add_style(topgroups)

,,val_binary_sharpe,val_binary_long_sharpe,val_binary_short_sharpe
asset,start,,,
dow_jones,340,0.268669,0.164516,0.212458
nasdaq_100,450,0.045737,0.015154,0.034036
russell_2000,440,-0.029127,-0.019701,-0.018956
sp_500,470,0.040615,0.026689,0.049068


In [148]:
res = []
for start in range(STUDY_START+10, STUDY_END, 10):
    r = range(start, start+10)
    #ad = range_diff(asset_pavg, r, base_range).groupby('asset').mean()
    ad = range_diff(asset_pavg, r, base_range).groupby('asset').mean() / range_diff(asset_pavg, r, base_range).groupby('asset').std()
    ad = ad.assign(start=start).set_index('start', append=True)
    res.append(ad)
res = pd.concat(res, axis=0).sort_index(level=[0, 1])
topgroups = res.loc[res.groupby(['asset'])['val_binary_sharpe'].idxmax()]
add_style(topgroups)

,,val_binary_sharpe,val_binary_long_sharpe,val_binary_short_sharpe
asset,start,,,
dow_jones,340,1.451202,1.315246,1.421927
nasdaq_100,450,0.174780,0.077072,0.174287
russell_2000,440,-0.075238,-0.074909,-0.065027
sp_500,470,0.109392,0.126277,0.145101


In [149]:
bestparams = {}
for group in topgroups.index:
    asset, start = group
    bestgroup = dv_adj.xs((asset, 'np')).loc[str(start):str(start+9), val_binary_sharpe]
    bestidx = bestgroup.loc[:, ['val_binary_long_sharpe', 'val_binary_short_sharpe']].mean(axis=1).idxmax()
    bestparams[asset] = int(bestidx)
    display(asset, add_style(bestgroup))

'dow_jones'

,val_binary_sharpe,val_binary_long_sharpe,val_binary_short_sharpe
params,,,
340,-0.398152,-0.294210,-0.556436
341,-0.316123,-0.237570,-0.491104
342,0.813245,0.453559,0.449202
343,0.254834,0.129632,-0.044765
344,0.472696,0.225723,0.194075
345,0.156607,0.030950,-0.078280
346,0.254553,0.268912,-0.238328
347,0.480467,0.227392,0.207066
348,-0.206676,-0.044712,-0.656661


'nasdaq_100'

,val_binary_sharpe,val_binary_long_sharpe,val_binary_short_sharpe
params,,,
450,0.314752,0.284958,-0.020197
451,0.354710,0.263179,0.033972
452,1.204628,0.970298,0.563002
453,0.974922,0.825145,0.398354
454,0.543549,0.385580,0.176481
455,1.177836,0.828488,0.628701
456,0.129880,0.222110,-0.261357
457,0.595367,0.523514,0.154919
458,0.469049,0.506578,-0.032201


'russell_2000'

,val_binary_sharpe,val_binary_long_sharpe,val_binary_short_sharpe
params,,,
440,0.912478,0.542772,0.738309
441,0.484336,0.206442,0.483299
442,0.892455,0.497064,0.765005
443,0.649577,0.310140,0.627086
444,0.452039,0.174389,0.490633
445,0.843829,0.420836,0.822444
446,0.608354,0.507767,0.354951
447,0.274042,0.053421,0.393296
448,0.474889,0.375981,0.291213


'sp_500'

,val_binary_sharpe,val_binary_long_sharpe,val_binary_short_sharpe
params,,,
470,0.034601,0.015707,-0.310031
471,0.977747,0.533390,0.547316
472,0.540012,0.313252,0.104721
473,-0.001982,-0.025032,-0.328563
474,0.549774,0.310314,0.123277
475,-0.592488,-0.423435,-0.797046
476,0.973783,0.803892,0.201736
477,0.384010,0.270556,-0.065063
478,0.601951,0.594000,-0.097592


In [151]:
bestparams

{'dow_jones': 342, 'nasdaq_100': 452, 'russell_2000': 440, 'sp_500': 471}

## Compare Results Over Val by Params

In [135]:
sel_param = lambda d, p: d.xs(p, level=2, drop_level=False)
sel_asset = lambda d, a: d.xs(a, level=0, drop_level=False)

In [136]:
sp_500 = sel_asset(dv_adj.loc[:, val_binary_sharpe], 'sp_500').sort_index(level=2)
russell_2000 = sel_asset(dv_adj.loc[:, val_binary_sharpe], 'russell_2000').sort_index(level=2)
nasdaq_100 = sel_asset(dv_adj.loc[:, val_binary_sharpe], 'nasdaq_100').sort_index(level=2)
dow_jones = sel_asset(dv_adj.loc[:, val_binary_sharpe], 'dow_jones').sort_index(level=2)

In [137]:
add_style(nasdaq_100)

In [138]:
add_style(gb_filter_monotonic(sp_500))

In [139]:
add_style(gb_filter_neg(gb_filter_monotonic_partial(sp_500)))

In [114]:
add_style(gb_filter_monotonic(russell_2000))

In [116]:
add_style(gb_filter_neg(gb_filter_monotonic_partial(russell_2000)))

In [ ]:
add_style(gb_filter_monotonic(nasdaq_100))

In [ ]:
add_style(gb_filter_neg(gb_filter_monotonic_partial(nasdaq_100)))

In [ ]:
add_style(gb_filter_monotonic(dow_jones))

In [ ]:
add_style(gb_filter_neg(gb_filter_monotonic_partial(dow_jones)))